<a href="https://colab.research.google.com/github/doris-asongafac/Support-Vector-Machines/blob/main/Exerxise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Using OpenAI's API (requires API key)
import openai

openai.api_key = "your-api-key-here"

def classify_emotion_gpt(review):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # or "gpt-4" for better results
            messages=[
                {"role": "system", "content": "You are an expert emotion classifier. Classify the movie review into exactly one of the following emotions: happiness, sadness, fear, disgust, anger, surprise. Only respond with the emotion name."},
                {"role": "user", "content": review}
            ],
            temperature=0
        )
        result = response['choices'][0]['message']['content'].strip().lower()

        # Normalize to your emotion categories
        emotion_mapping = {
            'happiness': 'happiness', 'happy': 'happiness', 'joy': 'happiness',
            'sadness': 'sadness', 'sad': 'sadness',
            'fear': 'fear', 'fearful': 'fear',
            'disgust': 'disgust', 'disgusted': 'disgust',
            'anger': 'anger', 'angry': 'anger',
            'surprise': 'surprise', 'surprised': 'surprise'
        }

        for key, value in emotion_mapping.items():
            if key in result:
                return value
        return "unknown"
    except Exception as e:
        print(f"Error: {e}")
        return "unknown"

In [ ]:
from transformers import pipeline

# Step 1: Load a high-quality sentiment classifier
sentiment_classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Step 2: Map sentiment to emotions based on review content
def two_step_emotion_classifier(review):
    # Get sentiment
    sentiment = sentiment_classifier(review)[0]
    sentiment_label = sentiment['label']
    confidence = sentiment['score']

    # Use high-confidence rules based on keywords
    review_lower = review.lower()

    # For positive sentiment
    if sentiment_label == "POSITIVE":
        if any(word in review_lower for word in ["amaz", "love", "great", "excellent", "enjoy"]):
            return "happiness"
        elif any(word in review_lower for word in ["surpris", "unexpect", "twist", "shock"]):
            return "surprise"
        else:
            return "happiness"  # Default for positive

    # For negative sentiment
    else:
        if any(word in review_lower for word in ["anger", "angry", "annoy", "frustrat"]):
            return "anger"
        elif any(word in review_lower for word in ["disgust", "repuls", "sicken", "gross"]):
            return "disgust"
        elif any(word in review_lower for word in ["scare", "fear", "terrif", "horror"]):
            return "fear"
        elif any(word in review_lower for word in ["sad", "depress", "disappoint", "heart"]):
            return "sadness"
        else:
            return "sadness"  # Default for negative

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load a sentence transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for your reviews
def create_embeddings(reviews):
    return embedding_model.encode(reviews)

# Setup training
X = create_embeddings(data['review'].tolist())
y = data['Final Emotion'].tolist()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'data' is not defined

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

def advanced_preprocess(text):
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Remove special characters but keep sentence structure
    text = re.sub(r'[^\w\s.,!?]', ' ', text)

    # Expand contractions
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'m", " am", text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join back into text
    text = ' '.join(words)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def ensemble_emotion_classifier(review):
    # Get predictions from multiple methods
    prediction1 = classifier1(review)  # Your first model
    prediction2 = classifier2(review)  # Your second model
    prediction3 = rule_based_classifier(review)  # Rule-based classifier

    # Vote for the most common prediction
    predictions = [prediction1, prediction2, prediction3]
    counts = {}
    for pred in predictions:
        if pred in counts:
            counts[pred] += 1
        else:
            counts[pred] = 1

    # Return the most common prediction
    return max(counts, key=counts.get)

In [ ]:
# Analyze distribution of emotions in your dataset
emotion_counts = data['Final Emotion'].value_counts()
print("Emotion distribution:")
print(emotion_counts)

# Check for class imbalance
if max(emotion_counts) / min(emotion_counts) > 10:
    print("Warning: Severe class imbalance detected")

# Check for unusual reviews
data['review_length'] = data['review'].apply(len)
short_reviews = data[data['review_length'] < 20]
if len(short_reviews) > 0:
    print(f"Warning: {len(short_reviews)} very short reviews detected")

NameError: name 'data' is not defined